# 📃항공사 지연 요인 분석 보고서

# *️⃣프로젝트 개요

분석 목적 : 항공사의 지연 요인(운항사, 관제, 기상)을 파악하여 개선 방향 제시   
데이터 출처 : 항공 운항 지연률 데이터 (총 33,468건)   
분석 대상 변수 : 
- pct_carrier_delay: 운항사 요인
- pct_atc_delay: 관제 요인
- pct_weather_delay: 기상 요인
- airline: 항공사명

In [7]:
# 폰트 준비

# 폰트 확인
import matplotlib.font_manager as fm
for font in fm.fontManager.ttflist:
    if 'Noto' in font.name :
        print(font.name)

import matplotlib.pyplot as plt #그래프(시각화) 그리기 위한 라이브러리 불러오기
import platform # 내 컴퓨터(운영체제) 정보를 알아내는 모듈 불러오기

plt.rc('font', family = 'Noto Sans KR')

Noto Sans KR
Noto Sans KR
Noto Sans KR
Noto Sans KR
Noto Sans KR
Noto Sans KR
Noto Sans KR


In [3]:
# 데이터 불러오기
import pandas as pd

df = pd.read_csv('../data/airline_stats.csv')
df


,pct_carrier_delay,pct_atc_delay,pct_weather_delay,airline
0,8.153226,1.971774,0.762097,American
1,5.959924,3.706107,1.585878,American
2,7.157270,2.706231,2.026706,American
3,12.100000,11.033333,0.000000,American
4,7.333333,3.365591,1.774194,American
...,...,...,...,...
33463,6.186422,8.798491,1.651940,Southwest
33464,9.522167,3.591133,0.261084,Southwest
33465,9.164179,2.664179,0.343284,Southwest
33466,5.152293,1.964520,0.122817,Southwest


# 0️⃣ 데이터 확인
   
**1. 데이터 구조 파악(데이터 확인)**   
- 파일의 구조(행, 열), 데이터 타입, 각 컬럼의 의미, 데이터의 크기(행 개수, 열 개수)를 파악   
   
**2. 결측치(Missing Value) 확인**   
   
**3. 이상치/오류 데이터 탐색**   
- 일반적인 범위를 벗어난 값이나 논리적으로 말이 안 되는 값(예: 나이가 -1살, 요금이 음수 등)이 있는지 확인한다.   
   
**4. 데이터 타입(Type) 확인 및 변환**   
- 숫자인데 문자열로 되어 있다든지, 날짜가 문자로 저장돼 있는 경우 적절하게 변환한다.   
   
**5. 중복 데이터 확인 및 제거**   
- 같은 행이 여러 번 반복돼 있는 경우가 있다면 중복을 제거한다.   
   
**6. 기본 통계치 확인**   
- 각 컬럼의 평균, 최대, 최소, 표준편차 등을 확인해 전체적인 분포를 파악한다.   
   
**7. 컬럼별 값의 분포 확인**   
- 범주형(카테고리) 컬럼이라면 각각 값이 얼마나 분포하는지 파악한다.
      
**8. 데이터의 목적에 맞는 정제/필터링**   
- 불필요한 컬럼 제거, 분석 목적에 맞는 데이터만 남기기 등 목적에 따라 정제한다.   

# 1️⃣ 데이터 구조 파악 

In [9]:
print(df.info())                            #데이터 타입 및 구조 파악
print(df.head(2))                           #데이터 컬럼 값 형태 파악
print(df.describe())                        #데이터 기본 통계 파악
print(df.select_dtypes(include=['number'])) # 수치형 데이터 파악

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33468 entries, 0 to 33467
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pct_carrier_delay  33440 non-null  float64
 1   pct_atc_delay      33440 non-null  float64
 2   pct_weather_delay  33440 non-null  float64
 3   airline            33468 non-null  object 
dtypes: float64(3), object(1)
memory usage: 1.0+ MB
None
   pct_carrier_delay  pct_atc_delay  pct_weather_delay   airline
0           8.153226       1.971774           0.762097  American
1           5.959924       3.706107           1.585878  American
       pct_carrier_delay  pct_atc_delay  pct_weather_delay
count       33440.000000   33440.000000       33440.000000
mean            7.041475       5.089962           0.684776
std             4.497596       4.718647           1.073743
min             0.000000       0.000000           0.000000
25%             4.145455       2.146565           0.00000

1. 33468행 4열
2. 수치열데이터 3개, 범주형데이터 1개
- pct_carrier_delay: 운항사 요인 - 결측치 있음 / 수치형 데이터
- pct_atc_delay: 관제 요인 - 결측치 있음 / 수치형 데이터
- pct_weather_delay: 기상 요인 - 결측치 있음 / 수치형 데이터
- airline: 항공사명 - 결측치 없음 / 범주형 데이터

std: 데이터들이 평균에서 얼마나 퍼져있는지(표준편차)

In [10]:
df['pct_carrier_delay'].describe()

count    33440.000000
mean         7.041475
std          4.497596
min          0.000000
25%          4.145455
50%          6.356893
75%          9.140474
max        100.000000
Name: pct_carrier_delay, dtype: float64

In [18]:
# 이상치 한번에 구하기

for col in df.select_dtypes(include=['int64','float64']):
    desc = df[col].describe()
    Q1 = desc['25%']
    Q3 = desc['75%']
    IQR = Q3 - Q1
    lower = Q1 - (1.5*IQR)
    upper = Q3 + (1.5*IQR)
    outliers = df[(df[col] < lower) | (df[col] > upper)]
    lowerliers = df[(df[col] < lower)]
    upperliers  = df[(df[col] > upper)]
    # print(lower)
    print(upper)
    # print(f'{col} 변수의 이상치 개수: {len(outliers)}')
    print(upperliers)

16.6330042455691
       pct_carrier_delay  pct_atc_delay  pct_weather_delay    airline
22             21.419355       4.709677           1.419355   American
199            25.000000      25.000000           0.000000      Delta
238            24.000000       9.333333           0.000000      Delta
279            50.000000       0.000000           0.000000      Delta
445            17.017857       2.357143           1.339286   American
...                  ...            ...                ...        ...
31850          17.550633       4.132911           1.569620  Southwest
32763          18.612903       4.483871           0.000000      Delta
33015          16.814815       1.814815           0.000000   American
33373          17.500000       5.800000           0.000000     United
33378          18.600000      18.300000           0.000000     United

[961 rows x 4 columns]
13.35574825816505
       pct_carrier_delay  pct_atc_delay  pct_weather_delay   airline
10              8.500000      15

In [ ]:
# 16.6330042455691 이건 항공사 이상치
#        pct_carrier_delay  pct_atc_delay  pct_weather_delay    airline
# 22             21.419355       4.709677           1.419355   American
# 199            25.000000      25.000000           0.000000      Delta
# 238            24.000000       9.333333           0.000000      Delta
# 279            50.000000       0.000000           0.000000      Delta
# 445            17.017857       2.357143           1.339286   American
# ...                  ...            ...                ...        ...
# 31850          17.550633       4.132911           1.569620  Southwest
# 32763          18.612903       4.483871           0.000000      Delta
# 33015          16.814815       1.814815           0.000000   American
# 33373          17.500000       5.800000           0.000000     United
# 33378          18.600000      18.300000           0.000000     United

# [961 rows x 4 columns]

# ---

# 13.35574825816505 이건 관제탑 이상치 
#        pct_carrier_delay  pct_atc_delay  pct_weather_delay   airline
# 10              8.500000      15.295455           2.318182  American
# 24             13.760369      45.115207           0.460829  American
# 33              8.419355      49.451613           0.000000  American
# 37              8.935484      29.516129           0.000000  American
# 46             12.161290      27.913978           0.784946  American
# ...                  ...            ...                ...       ...
# 33151          14.428571      15.333333           0.000000  Jet Blue
# 33210           3.173333      13.588889           0.640000     Delta
# ...
# 33361           4.977778       1.400000           2.400000    United
# 33379          13.937500       0.250000           6.250000    United

# [2181 rows x 4 columns]


In [ ]:
# 경고 메세지를 무시하고 출력되지 않게 만드는 코드 
# 경고 메세지가 너무 많아서, 결과 확인에 방해가 될 때 사용 

import warnings
warnings.filterwarnings('ignore')

In [6]:
# 범주형 변수의 값 목록
df['airline'].value_counts()

# 운행 9000회대 - Delta 항공
# 운행 5000회대 - American / Southwest / United
# 운행 3000회대 - Alaska / Jet Blue

airline
Delta        9107
American     5725
Southwest    5584
United       5426
Alaska       3851
Jet Blue     3775
Name: count, dtype: int64

In [10]:
# 항공사별 기술통계 비교
ndf = df.groupby('airline').describe().T
ndf

airline                       Alaska     American        Delta     Jet Blue  \
pct_carrier_delay count  3851.000000  5723.000000  9095.000000  3773.000000   
                  mean      3.521889     9.042180     6.333460     8.081843   
                  std       2.483224     4.139067     4.695292     3.797737   
                  min       0.000000     0.000000     0.000000     0.000000   
                  25%       1.935484     6.338538     3.813989     5.341176   
                  50%       3.225806     8.428571     5.548387     7.657895   
                  75%       4.685599    10.988763     7.821952    10.280000   
                  max      22.290323    50.000000   100.000000    28.000000   
pct_atc_delay     count  3851.000000  5723.000000  9095.000000  3773.000000   
                  mean      5.472672     5.993529     4.462978     7.345231   
                  std       5.699303     4.647170     4.364335     5.227372   
                  min       0.000000     0.000000     0.000000     0.000000   
                  25%       1.878614     3.108403     1.818182     3.784314   
                  50%       3.776860     4.986523     3.508108     6.136905   
                  75%       7.001330     7.594450     5.839901     9.612903   
                  max      54.838710    60.000000   100.000000    40.986111   
pct_weather_delay count  3851.000000  5723.000000  9095.000000  3773.000000   
                  mean      0.513497     1.122920     0.680765     0.465327   
                  std       1.136494     1.097811     1.093255     0.764147   
                  min       0.000000     0.000000     0.000000     0.000000   
                  25%       0.000000     0.296719     0.000000     0.000000   
                  50%       0.000000     0.884354     0.316406     0.157312   
                  75%       0.497395     1.641725     0.893346     0.666667   
                  max      14.633803    12.500000    20.500000     9.923077   

airline                    Southwest       United  
pct_carrier_delay count  5584.000000  5414.000000  
                  mean      7.521997     7.398833  
                  std       3.345085     5.369283  
                  min       0.000000     0.000000  
                  25%       5.070294     4.034708  
                  50%       6.960930     6.445210  
                  75%       9.354839     9.633110  
                  max      24.800000   100.000000  
pct_atc_delay     count  5584.000000  5414.000000  
                  mean      3.147019     5.348133  
                  std       2.513162     4.992464  
                  min       0.000000     0.000000  
                  25%       1.612527     2.431308  
                  50%       2.559242     4.353647  
                  75%       3.790421     7.030109  
                  max      21.542722   100.000000  
pct_weather_delay count  5584.000000  5414.000000  
                  mean      0.512611     0.680697  
                  std       0.649631     1.338433  
                  min       0.000000     0.000000  
                  25%       0.105675     0.000000  
                  50%       0.305598     0.263780  
                  75%       0.663306     0.861395  
                  max       6.240987    33.333333

# 2️⃣ 결측치 처리

방법 설명
지연 같은 경우는 극단적인 지연이 있을 수 있으므로 평균보다는 중앙값을 사용하는 게 좋음

In [11]:
df.isnull().sum()

pct_carrier_delay    28
pct_atc_delay        28
pct_weather_delay    28
airline               0
dtype: int64

In [17]:
num = df.select_dtypes('number')

df1 = df.fillna(num.median())

In [18]:
df1.isnull().sum()

pct_carrier_delay    0
pct_atc_delay        0
pct_weather_delay    0
airline              0
dtype: int64

# 3️⃣ 데이터분석 - EDA 

1. 히스토그램
- 항공사 간의 지연 유형에 따른 지연율과 횟수 비교

ex) 
0~10점 : 2명

10~20점 : 5명

20~30점 : 7명

In [ ]:
import seaborn as sns
#항공사 리스트
air = df['airline'].unique()

#지연 유형 컬럼들
delay_cols = ['pct_carrier_delay','pct_atc_delay', 'pct_weather_delay']

plt.figure()

#지연유형별 subplot
for i, col in enumerate(delay_cols, start=1):
    plt.subplot(1,3,i)
    for airl in air:
        air1.hist()
        



['pct_carrier_delay', 'pct_atc_delay', 'pct_weather_delay']


4️⃣5️⃣6️⃣7️⃣8️⃣9️⃣🔟

